#TAG SMART EMAILS

GENSIM AND LDA

https://radimrehurek.com/gensim/apiref.html

In [0]:
#Library
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import spacy
import pandas as pd

In [0]:
%pwd

'/content'

In [0]:
import requests
url = 'https://raw.githubusercontent.com/Moly-malibu/Tag-Smart-Emails/master/Enronemails.csv?token=AN2RW2GO4AWTXK4OPUDXJV26ZQ3QC'
myfile = requests.get(url)
open('email.csv', 'wb').write(myfile.content)

24004915

In [0]:
#EMAILS FROM ENRON COMPANY SAMPLING
df = pd.read_csv("email.csv")
print(df.shape) 
df.head(7).drop(['Unnamed: 0'], axis=1)

(10000, 3)


,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...
5,allen-p/_sent_mail/1002.,Message-ID: <30965995.1075863688265.JavaMail.e...
6,allen-p/_sent_mail/1003.,Message-ID: <16254169.1075863688286.JavaMail.e...


In [0]:
#TO GET EMAIL BODY
[s.strip() for s in df['message'][1].splitlines()]

['Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>',
 'Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)',
 'From: phillip.allen@enron.com',
 'To: john.lavorato@enron.com',
 'Subject: Re:',
 'Mime-Version: 1.0',
 'Content-Type: text/plain; charset=us-ascii',
 'Content-Transfer-Encoding: 7bit',
 'X-From: Phillip K Allen',
 'X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>',
 'X-cc:',
 'X-bcc:',
 "X-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail",
 'X-Origin: Allen-P',
 'X-FileName: pallen (Non-Privileged).pst',
 '',
 'Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.',
 '',
 'As far as the business meetings, I think it would be more productive to try and stimulate discussions across

In [0]:
email = ' '.join([s.strip() for s in df['message'][1].splitlines()][15:])

In [0]:
#CLEAN EMAIL BODY
def cleanemail(email):
    return ' '.join([s.strip() for s in email.splitlines()][15:])

df['email_body'] = df['message'].apply(cleanemail)

df.head()

,Unnamed: 0,file,message,email_body
0,0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes th...
2,2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go!!!
3,3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the sal..."
4,4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Let's shoot for Tuesday at 11:45.


In [0]:
#CLEAN HTML
import re
import html

def cleanup_html(raw_html):
    clean_html = re.sub(r'<.*?>', '', raw_html)
    clean_html_http = re.sub(r'http\S+([\.]{3})?', '', clean_html)
    clean_txt = html.unescape(clean_html)
    return clean_txt
df['email_body'] = df['email_body'].apply(cleanup_html)
df.sample(10)
for row in df['email_body'].sample(10):
    print(row)
    print()

X-bcc: X-Folder: \Phillip_Allen_Dec2000\Notes Folders\Notes inbox X-Origin: Allen-P X-FileName: pallen.nsf  FYI. ---------------------- Forwarded by Rebecca W Cantrell/HOU/ECT on 12/13/2000 04:18 PM ---------------------------   "Randall Rich"  on 12/13/2000 04:13:55 PM To: "Jeffrey Watkiss" , , , , , ,  cc: Subject: Final Filed Version   The filed version of the comments in the San Diego Gas & Electric matter at FERC is attached.  - SANDIEGO.DOC

 I would look at properties in San Antonio or Dallas.

X-Folder: \SBAILE2 (Non-Privileged)\Bailey, Susan\Sent Items X-Origin: Bailey-S X-FileName: SBAILE2 (Non-Privileged).pst   All,  Attached is the template for Annex B & B-1, along with the LC form for the captioned Counterparty's Omnibus Confirmation for Deal No. YK1853.1.   Contact me with any questions.   Regards, Susan S. Bailey Senior Legal Specialist Enron Wholesale Services Legal Department 1400 Smith Street, Suite 3803A Houston, Texas 77002 phone: (713) 853-4737 fax: (713) 646-3490 

In [0]:
#CLEAN DATA
#remove whitespace
df['email_body'] = df['email_body'].str.strip().str.lower()

#Start with date
df['email_body'].str.match('\d?\d/\d?\d/\d{4}').all()

#Replace all non-overlapping matches
df['email_body'] = df['email_body'].str.replace('[^a-zA-Z\s]', '').str.replace('\s+', ' ')

In [0]:
import nltk
import string
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
#Getting Stopwords
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

# define additional stop words
import nltk
import spacy

nlp = spacy.load('en_core_web_sm')
my_stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 
                 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 
                 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 
                 "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 
                 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 
                 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 
                 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 
                 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 
                 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 
                 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 
                 'into', 'through', 'during', 'before', 'after', 'above', 'below',
                 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 
                 'again', 'further', 'then', 'once', 'here', 'there', 'when', 
                 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 
                 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 
                 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 
                 'can', 'will', 'just', 'don', "don't", 'should', "should've", 
                 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 
                 "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', 
                 "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', 
                 "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 
                 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 
                 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
                 "weren't", 'won', "won't", 'wouldn', "wouldn't"]

STOP_WORDS = nlp.Defaults.stop_words.union(my_stop_words)

stop = stopwords.words('english')
sno = SnowballStemmer('english')
print (stop)
print('***************************************')
print(sno.stem('tasty'))

STOP_WORDS = list(STOP_WORDS)
print('\n'.join([word for word in STOP_WORDS[:10]]))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:
tokens = []
for doc in nlp.pipe(df['email_body'], batch_size=500):

    doc_tokens = []

    for token in doc:
        if (token.is_stop == False) & (token.is_punct == False):
            doc_tokens.append(token.lemma_.lower())

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [0]:
df['tokens'].head()

0                                           [forecast]
1    [travel, business, meeting, take, fun, trip, e...
2                              [test, successful, way]
3    [randy, send, schedule, salary, level, schedul...
4                                [let, shoot, tuesday]
Name: tokens, dtype: object

In [0]:
id2word = Dictionary(df['tokens'])
print(len(id2word))

45828


In [0]:
id2word.filter_extremes(no_below=5, no_above=.98)
print(len(id2word))

14979


In [0]:
#https://radimrehurek.com/gensim/corpora/dictionary.html

corpus = [id2word.doc2bow(d) for d in df['tokens']]

In [0]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   random_state=723812,
                   num_topics = 15,
                   passes=10,
                   workers=12
                  )

In [0]:
model = LdaMulticore(corpus=corpus, num_topics=10, id2word=id2word, workers=12, passes=5)

In [0]:
import re

words = [re.findall(r'"([^"]*)"',t[1]) for t in model.print_topics()]

In [0]:
topics = [' '.join(t[0:5]) for t in words]

In [0]:
for id, t in enumerate(topics): 
    print(f"------ Topic {id} ------")
    print(t, end="\n")

------ Topic 0 ------
request email cost market iso
------ Topic 1 ------
enron email pm send subject
------ Topic 2 ------
buy day wine enron price
------ Topic 3 ------
market power price enron say
------ Topic 4 ------
enron say company energy power
------ Topic 5 ------
image company enron message gas
------ Topic 6 ------
pm subject time send outage
------ Topic 7 ------
phillip john subject pm k
------ Topic 8 ------
john subject pm send message
------ Topic 9 ------
email gas spread price chart


In [0]:
!pip install pyLDAvis
import pyLDAvis

     |████████████████████████████████| 1.6MB 9.8MB/s 
     |████████████████████████████████| 552kB 54.5MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=e578b6a84db1371a88ff0256f5015b1d7c0e278e22d95bbbfd83322d2b08d9d7
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=cb12e59f211c20a11d44c3b0e0a24af2ed30aefe8ad04046db6f2c1740e9910f
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [0]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

pyLDAvis.gensim.prepare(model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.116310  0.049421       1        1  21.221546
8      0.054948 -0.005994       2        1  17.427231
7      0.183789  0.026966       3        1  13.102701
3     -0.039487  0.027966       4        1  10.887095
0     -0.060642 -0.060082       5        1  10.115372
5     -0.035967  0.049277       6        1   5.953598
6      0.043070 -0.025051       7        1   5.639210
2     -0.025084 -0.116048       8        1   5.594696
9     -0.026804  0.054403       9        1   5.228474
1      0.022488 -0.000859      10        1   4.830077, topic_info=           Term         Freq        Total Category  logprob  loglift
173          pm  8200.000000  8200.000000  Default  30.0000  30.0000
2682      image  3027.000000  3027.000000  Default  29.0000  29.0000
962       email  5846.000000  5846.000000  Default  28.0000  28.0000
77         john  7569.000000  7569.000000  Default  27.0000  27.0000
193     subject  7832.000000  7832.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
1682  xfilename   152.545563  1559.584839  Topic10  -5.9311   0.7056
325     include   160.588043  2287.725098  Topic10  -5.8798   0.3738
340        know   158.184372  2985.475830  Topic10  -5.8948   0.0926
174       power   161.311600  4119.846191  Topic10  -5.8753  -0.2099
133      energy   155.874878  3599.017090  Topic10  -5.9096  -0.1091

[922 rows x 6 columns], token_table=      Topic      Freq Term
term                      
90        1  0.022667     
90        2  0.206431     
90        3  0.396672     
90        4  0.043715     
90        5  0.068001     
...     ...       ...  ...
6314      6  0.016287  zip
6314      7  0.146587  zip
6314      8  0.016287  zip
6314      9  0.024431  zip
6314     10  0.016287  zip

[4971 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 9, 8, 4, 1, 6, 7, 3, 10, 2])

In [0]:
#'corpus'

distro = [lda[d] for d in corpus]

num_topics = 20

def update(doc):
        d_dist = {k:0 for k in range(0,num_topics)}
        for topic in doc:
            d_dist[topic[0]] = topic[1]
        return d_dist
    
new_distro = [update(d) for d in distro]

In [0]:
#Correlated
topics = pd.DataFrame.from_records(new_distro)
topics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.533333,0,0,0,0,0
1,0.000000,0.000000,0.000000,0.102773,0.000000,0.000000,0.000000,0.000000,0.882783,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
2,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766666,0.016667,0.016667,0.016667,0.016667,0,0,0,0,0
3,0.000000,0.000000,0.637493,0.308340,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,0,0,0
4,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.016667,0.766666,0.016667,0.016667,0,0,0,0,0


In [0]:
df['email_body'] = topics.idxmax(axis=1)

In [0]:
#Topic  
df['email_body'].describe()

count    75.000000
mean      5.453333
std       4.037973
min       0.000000
25%       2.000000
50%       5.000000
75%       8.000000
max      14.000000
Name: email_body, dtype: float64

In [0]:
df['email_body'].value_counts()

1     14
3     13
5     12
8     10
9      5
14     3
13     3
12     3
2      3
0      3
10     2
11     1
7      1
6      1
4      1
Name: email_body, dtype: int64

##Create the Emails Tags

##tfidf term frequency–inverse document frequency


In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
from nltk.tokenize import word_tokenize

# split into words
tokens = word_tokenize(email)

# convert to lower case
tokens = [w.lower() for w in tokens]

# prepare regex for char filtering
re_punc = re.compile('[%s]' % re.escape(string.punctuation))

# remove punctuation from each word
stripped = [re_punc.sub('', w) for w in tokens]

# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]

# filter out stop words
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
print(words[:200])

['traveling', 'business', 'meeting', 'takes', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'business', 'plan', 'meetings', 'take', 'trip', 'without', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'get', 'honest', 'opinions', 'whether', 'trip', 'even', 'desired', 'necessary', 'far', 'business', 'meetings', 'think', 'would', 'productive', 'try', 'stimulate', 'discussions', 'across', 'different', 'groups', 'working', 'often', 'presenter', 'speaks', 'others', 'quiet', 'waiting', 'turn', 'meetings', 'might', 'better', 'held', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'austin', 'play', 'golf', 'rent', 'ski', 'boat', 'jet', 'ski', 'flying', 'somewhere', 'takes', 'much', 'time']


In [0]:
from sklearn.feature_extraction.text import CountVectorizer

count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=None)

count_train = count_vec.fit(words)
bag_of_words = count_vec.transform(words)

print(count_vec.get_feature_names())

['austin', 'better', 'boat', 'business', 'desired', 'different', 'discussion', 'discussions', 'especially', 'far', 'flying', 'formal', 'format', 'fun', 'golf', 'groups', 'held', 'holding', 'honest', 'jet', 'meeting', 'meetings', 'necessary', 'opinions', 'plan', 'play', 'prepare', 'presentation', 'presenter', 'productive', 'quiet', 'rent', 'round', 'ski', 'speaks', 'stimulate', 'suggest', 'suggestion', 'table', 'takes', 'think', 'time', 'traveling', 'trip', 'try', 'turn', 'waiting', 'working']


In [0]:
text_freque = TfidfVectorizer(smooth_idf=False, sublinear_tf=False, norm=None, analyzer='word')
txt_fitted = text_freque.fit(words)
txt_transformed = txt_fitted.transform(words)
print ("The text: ", words)

The text:  ['traveling', 'business', 'meeting', 'takes', 'fun', 'trip', 'especially', 'prepare', 'presentation', 'would', 'suggest', 'holding', 'business', 'plan', 'meetings', 'take', 'trip', 'without', 'formal', 'business', 'meetings', 'would', 'even', 'try', 'get', 'honest', 'opinions', 'whether', 'trip', 'even', 'desired', 'necessary', 'far', 'business', 'meetings', 'think', 'would', 'productive', 'try', 'stimulate', 'discussions', 'across', 'different', 'groups', 'working', 'often', 'presenter', 'speaks', 'others', 'quiet', 'waiting', 'turn', 'meetings', 'might', 'better', 'held', 'round', 'table', 'discussion', 'format', 'suggestion', 'go', 'austin', 'play', 'golf', 'rent', 'ski', 'boat', 'jet', 'ski', 'flying', 'somewhere', 'takes', 'much', 'time']


In [0]:
idfreque = text_freque.idf_
print(dict(zip(txt_fitted.get_feature_names(), idfreque)))

{'across': 5.31748811353631, 'austin': 5.31748811353631, 'better': 5.31748811353631, 'boat': 5.31748811353631, 'business': 3.9311937524164198, 'desired': 5.31748811353631, 'different': 5.31748811353631, 'discussion': 5.31748811353631, 'discussions': 5.31748811353631, 'especially': 5.31748811353631, 'even': 4.624340932976365, 'far': 5.31748811353631, 'flying': 5.31748811353631, 'formal': 5.31748811353631, 'format': 5.31748811353631, 'fun': 5.31748811353631, 'get': 5.31748811353631, 'go': 5.31748811353631, 'golf': 5.31748811353631, 'groups': 5.31748811353631, 'held': 5.31748811353631, 'holding': 5.31748811353631, 'honest': 5.31748811353631, 'jet': 5.31748811353631, 'meeting': 5.31748811353631, 'meetings': 3.9311937524164198, 'might': 5.31748811353631, 'much': 5.31748811353631, 'necessary': 5.31748811353631, 'often': 5.31748811353631, 'opinions': 5.31748811353631, 'others': 5.31748811353631, 'plan': 5.31748811353631, 'play': 5.31748811353631, 'prepare': 5.31748811353631, 'presentation': 5

In [0]:
text_freque.vocabulary_

{'across': 0,
 'austin': 1,
 'better': 2,
 'boat': 3,
 'business': 4,
 'desired': 5,
 'different': 6,
 'discussion': 7,
 'discussions': 8,
 'especially': 9,
 'even': 10,
 'far': 11,
 'flying': 12,
 'formal': 13,
 'format': 14,
 'fun': 15,
 'get': 16,
 'go': 17,
 'golf': 18,
 'groups': 19,
 'held': 20,
 'holding': 21,
 'honest': 22,
 'jet': 23,
 'meeting': 24,
 'meetings': 25,
 'might': 26,
 'much': 27,
 'necessary': 28,
 'often': 29,
 'opinions': 30,
 'others': 31,
 'plan': 32,
 'play': 33,
 'prepare': 34,
 'presentation': 35,
 'presenter': 36,
 'productive': 37,
 'quiet': 38,
 'rent': 39,
 'round': 40,
 'ski': 41,
 'somewhere': 42,
 'speaks': 43,
 'stimulate': 44,
 'suggest': 45,
 'suggestion': 46,
 'table': 47,
 'take': 48,
 'takes': 49,
 'think': 50,
 'time': 51,
 'traveling': 52,
 'trip': 53,
 'try': 54,
 'turn': 55,
 'waiting': 56,
 'whether': 57,
 'without': 58,
 'working': 59,
 'would': 60}

In [0]:
words_dist = dict(zip(txt_fitted.get_feature_names(), idfreque))
words_dist

{'across': 5.31748811353631,
 'austin': 5.31748811353631,
 'better': 5.31748811353631,
 'boat': 5.31748811353631,
 'business': 3.9311937524164198,
 'desired': 5.31748811353631,
 'different': 5.31748811353631,
 'discussion': 5.31748811353631,
 'discussions': 5.31748811353631,
 'especially': 5.31748811353631,
 'even': 4.624340932976365,
 'far': 5.31748811353631,
 'flying': 5.31748811353631,
 'formal': 5.31748811353631,
 'format': 5.31748811353631,
 'fun': 5.31748811353631,
 'get': 5.31748811353631,
 'go': 5.31748811353631,
 'golf': 5.31748811353631,
 'groups': 5.31748811353631,
 'held': 5.31748811353631,
 'holding': 5.31748811353631,
 'honest': 5.31748811353631,
 'jet': 5.31748811353631,
 'meeting': 5.31748811353631,
 'meetings': 3.9311937524164198,
 'might': 5.31748811353631,
 'much': 5.31748811353631,
 'necessary': 5.31748811353631,
 'often': 5.31748811353631,
 'opinions': 5.31748811353631,
 'others': 5.31748811353631,
 'plan': 5.31748811353631,
 'play': 5.31748811353631,
 'prepare': 5

In [0]:
import numpy as np

# get feature names
feature_names = np.array(text_freque.get_feature_names())
sorted_by_idf = np.argsort(text_freque.idf_)
print("Features with lowest id frequence:\n{}".format(feature_names[sorted_by_idf[:3]]))
print("\nFeatures with highest id frequence:\n{}".format(feature_names[sorted_by_idf[-3:]]))

Features with lowest id frequence:
['business' 'meetings' 'would']

Features with highest id frequence:
['often' 'get' 'opinions']


##TFID VECTORIZER  - Transforms text to feature vectors


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
data = tfidf.fit_transform(words) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
data = pd.DataFrame(data.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
data.head()

,austin,better,boat,business,desired,different,discussion,discussions,especially,far,flying,formal,format,fun,golf,groups,held,holding,honest,jet,meeting,meetings,necessary,opinions,plan,play,prepare,presentation,presenter,productive,quiet,rent,round,ski,speaks,stimulate,suggest,suggestion,table,takes,think,time,traveling,trip,try,turn,waiting,working
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#TOKENIZER AND LEMMA
def tokenize(document):
    
    doc = nlp(document)
    
    return [token.lemma_.strip() for token in doc if (token.is_stop != True) and (token.is_punct != True)]

In [0]:
# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        tokenizer=tokenize)

# Create a vocabulary and get word counts per document
data = tfidf.fit_transform(words) # Similiar to fit_predict

# Print word counts

# Get feature names to use as dataframe column headers
data = pd.DataFrame(data.todense(), columns=tfidf.get_feature_names())

# View Feature Matrix as DataFrame
data.head()

,business,meeting,trip
0,0.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,1.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0


In [0]:
# Calculate Distance of TF-IDF Vectors
from sklearn.metrics.pairwise import cosine_similarity

dist_matrix  = cosine_similarity(dtm)

In [0]:
# Turn it into a DataFrame

df = pd.DataFrame(dist_matrix)

In [0]:
words[5][:300]

'trip'

In [0]:
print(words[12][:130])

business


In [0]:
sum(data['business'])

4.0

##Json and CSV

In [0]:
def save_data_frame_as_csv(df=None, file=None):
    """
    Save a Pandas DataFrame to CSV format.
    """
    try:
        if not file.endswith('.csv'):
            file += '.csv'
        df.to_csv(file, index=False)
        print(f'DataFrame saved as: {file}')
    except Exception as e:
        print("DataFrame couldn't be saved: ", sys.exc_info()[0])
        raise

In [0]:
save_data_frame_as_csv(df, './merged_data.csv')

In [0]:
def save_data_frame_as_json(df=None, file=None, orient='records'):
  """
  Save a Pandas DataFrame to Json format.
  """
  try:
    if not file.endswith('.json'):
      file += '.json'
    df.to_json(file, orient=orient)
    print(f'Dataframe saved as: {file}')
  except Exception as e:
    print("Dataframe couldn't be saved: ", sys.exc_info()[0])
    raise

In [0]:
save_data_frame_as_json(df, '.merged_data.json', orient='records')